This notebook explores using scikit_hts package to implement optimal forecast reconciliation for hierarchical time series. <br>
Reference: https://towardsdatascience.com/optimal-forecast-reconciliation-for-hierarchical-time-series-ea892ca105a9

In [1]:
!pip install scikit-hts

Defaulting to user installation because normal site-packages is not writeable


### try scikit_hts with a naive hts (1 observation for n hierarchies)

In [2]:
import numpy as np
import pandas as pd 
import hts  # To install: pip install scikit-hts
import collections
from scipy.optimize import lsq_linear

In [3]:
# a naive hts
hts_df = pd.DataFrame([{'total': 2, 
                        'CA': 1.4, 'TX': 1.8, 'WI': 1.9, 
                        'CA_1': 0.8, 'CA_2': 0.6, 'CA_3': 0.9, 'CA_4': 0.3,
                        'TX_1': 0.03, 'TX_2': 0.5, 'TX_3': 0.5, 
                        'WI_1': 1.6, 'WI_2': 1.2, 'WI_3': 1.5
                        }])

In [4]:
hts_df

,total,CA,TX,WI,CA_1,CA_2,CA_3,CA_4,TX_1,TX_2,TX_3,WI_1,WI_2,WI_3
0,2,1.4,1.8,1.9,0.8,0.6,0.9,0.3,0.03,0.5,0.5,1.6,1.2,1.5


In [5]:
states = ['CA', 'TX', 'WI']
stores = ['CA_1', 'CA_2', 'CA_3', 'CA_4', 'TX_1', 'TX_2', 'TX_3', 'WI_1', 'WI_2', 'WI_3']

# build the hierarchy tree as a dictionary

total = {'total': list(states)}
state_h = {k: [v for v in stores if v.startswith(k)] for k in states}
hierarchy = {**total, **state_h}

In [6]:
hierarchy

{'total': ['CA', 'TX', 'WI'],
 'CA': ['CA_1', 'CA_2', 'CA_3', 'CA_4'],
 'TX': ['TX_1', 'TX_2', 'TX_3'],
 'WI': ['WI_1', 'WI_2', 'WI_3']}

In [7]:
tree = hts.hierarchy.HierarchyTree.from_nodes(nodes=hierarchy, df=hts_df)
sum_mat, sum_mat_labels = hts.functions.to_sum_mat(tree)

In [8]:
tree

- total
   |- CA
   |  |- CA_1
   |  |- CA_2
   |  |- CA_3
   |  - CA_4
   |- TX
   |  |- TX_1
   |  |- TX_2
   |  - TX_3
   - WI
      |- WI_1
      |- WI_2
      - WI_3

In [9]:
pred_dict = collections.OrderedDict()
for label in sum_mat_labels:
    pred_dict[label] = pd.DataFrame(data=hts_df[label].values, columns=['yhat'])

In [10]:
# perform forecast reconciliation
revised = hts.functions.optimal_combination(pred_dict, sum_mat, method='OLS', mse={})
revised_forecasts = pd.DataFrame(data=revised,
                                 index=hts_df.index,
                                 columns=sum_mat_labels)
print(revised_forecasts)

      total        WI        TX        CA      CA_1      CA_2      CA_3  \
0  3.135606  1.648295  0.755795  0.731515  0.332879  0.132879  0.432879   

       CA_4      TX_1      TX_2      TX_3      WI_1      WI_2      WI_3  
0 -0.167121 -0.061402  0.408598  0.408598  0.716098  0.316098  0.616098  


In [11]:
#  For non-negative reconciled forecasts
hat_mat = hts.functions.y_hat_matrix(pred_dict)
revised_nnls = np.dot(sum_mat, lsq_linear(sum_mat, hat_mat.flatten(), bounds=(0, np.inf))['x'])
revised_forecasts_nnls = pd.DataFrame(data=revised_nnls.reshape(1,-1),
                                 index=hts_df.index,
                                 columns=sum_mat_labels)

In [12]:
print(revised_forecasts_nnls)

      total        WI        TX        CA      CA_1      CA_2      CA_3  \
0  3.155263  1.633553  0.763158  0.758553  0.286184  0.086184  0.386184   

           CA_4          TX_1      TX_2      TX_3      WI_1      WI_2  \
0  9.425493e-22  5.896338e-17  0.381579  0.381579  0.711184  0.311184   

       WI_3  
0  0.611184  


### try hts with 28 observations at different hierarchies

In [13]:
# load m5 dataset
m5_input_path = "/ssd003/projects/forecasting_bootcamp/bootcamp_datasets/m5-forecasting-accuracy"
sell_price = pd.read_csv(f'{m5_input_path}/sell_prices.csv')
calendar = pd.read_csv(f'{m5_input_path}/calendar.csv')
train = pd.read_csv(f'{m5_input_path}/sales_train_evaluation.csv').set_index('id')
sample_sub = pd.read_csv(f'{m5_input_path}/sample_submission.csv')

In [14]:
cat_cols = ['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']
ts_cols = [col for col in train.columns if col not in cat_cols]
ts_dict = {t: int(t[2:]) for t in ts_cols}

all_sales = pd.DataFrame(train[ts_cols].sum()).transpose()
all_sales['id_str'] = 'all'
all_sales = all_sales[ ['id_str'] +  [c for c in all_sales if c not in ['id_str']] ]

state_sales = train.groupby('state_id',as_index=False)[ts_cols].sum()
state_sales['id_str'] = state_sales['state_id'] 
state_sales = state_sales[ ['id_str'] +  [c for c in state_sales if c not in ['id_str']] ]
state_sales = state_sales.drop(['state_id'],axis=1)

store_sales = train.groupby('store_id',as_index=False)[ts_cols].sum()
store_sales['id_str'] = store_sales['store_id'] 
store_sales = store_sales[ ['id_str'] +  [c for c in store_sales if c not in ['id_str']] ]
store_sales = store_sales.drop(['store_id'],axis=1)

In [15]:
aggregates = pd.concat([all_sales,state_sales,store_sales],ignore_index=True)

In [16]:
true_last_28 = aggregates.set_index('id_str').transpose().rename(columns={'all':'total'}).iloc[-28:]

In [17]:
true_last_28.head()

id_str,total,CA,TX,WI,CA_1,CA_2,CA_3,CA_4,TX_1,TX_2,TX_3,WI_1,WI_2,WI_3
d_1914,38793,17524,10662,10607,4472,3926,6359,2767,3076,3883,3703,3166,4178,3263
d_1915,35487,15012,9933,10542,3703,3525,5289,2495,2853,3502,3578,3194,4148,3200
d_1916,34445,14836,9575,10034,3715,3527,5065,2529,2984,3256,3335,3267,3805,2962
d_1917,34732,14664,9655,10413,3618,3754,5015,2277,2664,3441,3550,3201,4342,2870
d_1918,42896,17180,12162,13554,4573,4382,5705,2520,3687,4023,4452,4143,5719,3692


In [18]:
# add noise to true sales numbers to get synthetic prediction data
mu, sigma = 1000, 2000
noise = np.random.normal(mu, sigma, [true_last_28.shape[0], true_last_28.shape[1]]) 
synthetic_pred_28 = true_last_28 + noise

In [19]:
synthetic_pred_28.head()

id_str,total,CA,TX,WI,CA_1,CA_2,CA_3,CA_4,TX_1,TX_2,TX_3,WI_1,WI_2,WI_3
d_1914,40195.648511,19073.165087,12625.462145,11747.751611,8004.025604,8195.998320,7633.250087,943.409850,2752.555463,979.008336,10967.474479,3841.188162,6953.955116,2059.075831
d_1915,35999.677087,16723.461983,8966.150049,9561.330438,7839.383478,6748.507668,9022.548115,-333.941127,733.915443,4679.992173,3717.925485,3967.815237,4588.798893,5155.497897
d_1916,35556.399753,16468.431343,11542.722568,7260.633879,6847.007692,6058.970354,6951.260992,3330.966478,5114.506558,6754.670611,3249.789993,5544.131338,7745.111917,5768.868083
d_1917,39220.397736,15630.740009,8503.616208,12015.259618,4226.190946,6263.547449,1362.093459,5246.265887,4672.426541,8576.886584,6216.446872,3396.463231,6070.037993,3961.424021
d_1918,42573.951481,19302.337108,13342.276362,12342.599561,5255.544323,7705.443181,8698.898448,2045.938824,8277.331976,6290.984862,2301.515189,2141.520421,4721.175228,6608.279958


In [20]:
tree = hts.hierarchy.HierarchyTree.from_nodes(nodes=hierarchy, df=synthetic_pred_28)
sum_mat, sum_mat_labels = hts.functions.to_sum_mat(tree)

In [21]:
pred_dict = collections.OrderedDict()
for label in sum_mat_labels:
    pred_dict[label] = pd.DataFrame(data=synthetic_pred_28[label].values, columns=['yhat'])

In [22]:
revised = hts.functions.optimal_combination(pred_dict, sum_mat, method='OLS', mse={})
revised_forecasts = pd.DataFrame(data=revised,
                                 index=synthetic_pred_28.index,
                                 columns=sum_mat_labels)

In [23]:
revised_forecasts
# we could see there are negative forecast values, e.g. CA_4 at day_1915, 1916, 1917

,total,WI,TX,CA,CA_1,CA_2,CA_3,CA_4,TX_1,TX_2,TX_3,WI_1,WI_2,WI_3
d_1914,41767.298509,10845.630986,11965.118679,18956.548843,6548.991850,6740.964566,6178.216333,-511.623905,1841.248930,67.701803,10056.167946,3171.658788,6284.425742,1389.546457
d_1915,36496.946412,10226.073841,8634.618818,17636.253753,6429.322383,5338.446573,7612.487020,-1744.002222,568.177348,4514.254079,3552.187390,2805.802509,3426.786164,3993.485168
d_1916,37042.088657,9095.736565,11322.517038,16623.835054,5205.915076,4417.877739,5310.168377,1689.873862,3849.023184,5489.187236,1984.306618,2223.339747,4424.320326,2448.076492
d_1917,39316.274099,12296.518752,11172.244883,15847.510465,3913.544127,5950.900630,1049.446640,4933.619068,1907.921503,5812.381546,3451.941834,3019.327734,5692.902495,3584.288523
d_1918,43924.843000,11611.524933,13210.996639,19102.321427,4104.668486,6554.567344,7548.022611,895.062987,7057.720180,5071.373066,1081.903393,1521.703530,4101.358337,5988.463067
d_1919,53210.710811,16812.525729,14361.369668,22036.815415,3947.204166,8694.376904,6532.655433,2862.578911,4809.266181,7001.257417,2550.846070,4497.034664,6797.997994,5517.493071
d_1920,57251.137804,14469.669747,15525.059838,27256.408219,7037.208226,6545.999587,8125.382530,5547.817877,5690.945284,4788.618219,5045.496335,5693.077646,2400.692780,6375.899321
d_1921,45279.563834,17772.314206,9762.783331,17744.466297,6360.173697,5688.853247,4302.156844,1393.282509,3905.204201,1114.190154,4743.388976,4080.069967,6857.610844,6834.633395
d_1922,48099.315383,15553.070936,12864.786510,19681.457937,5581.061256,4280.113461,6425.602915,3394.680305,6834.809378,4137.740930,1892.236202,3109.928570,6335.587783,6107.554583
d_1923,41610.985301,11790.359851,11031.514786,18789.110664,5296.614595,3129.194670,7594.406636,2768.894763,5112.837359,2725.130539,3193.546888,2693.789767,4802.849178,4293.720906


In [24]:
# add non-negative constraint when reconciliate
hat_mat = hts.functions.y_hat_matrix(pred_dict)

In [25]:
b_mat = []
for i in range(28):
    b_mat.append(lsq_linear(sum_mat, hat_mat[i].flatten(), bounds=(0, np.inf))['x'])

In [26]:
b_mat_final = np.stack(b_mat).transpose()
revised_nnls = np.dot(sum_mat, b_mat_final).transpose()
revised_forecasts_nnls = pd.DataFrame(data=revised_nnls,
                                 index=synthetic_pred_28.index,
                                 columns=sum_mat_labels)

In [27]:
revised_forecasts_nnls
# the previously negative forecasts are now close to 0 (CA_$ at d_1915, d_1916, d_1917)

,total,WI,TX,CA,CA_1,CA_2,CA_3,CA_4,TX_1,TX_2,TX_3,WI_1,WI_2,WI_3
d_1914,41806.654194,10816.114223,11935.601916,19054.938056,6411.246952,6603.219669,6040.471435,3.104512e-23,1831.410009,57.862882,10046.329025,3161.819866,6274.586820,1379.707536
d_1915,36631.100429,10125.458329,8534.003305,17971.638796,5959.783323,4868.907513,7142.947960,9.305782e-22,534.638844,4480.715574,3518.648886,2772.264005,3393.247660,3959.946664
d_1916,37042.088657,9095.736565,11322.517038,16623.835054,5205.915076,4417.877739,5310.168377,1.689874e+03,3849.023184,5489.187236,1984.306618,2223.339747,4424.320326,2448.076492
d_1917,39316.274099,12296.518752,11172.244883,15847.510465,3913.544127,5950.900630,1049.446640,4.933619e+03,1907.921503,5812.381546,3451.941834,3019.327734,5692.902495,3584.288523
d_1918,43924.843000,11611.524933,13210.996639,19102.321427,4104.668486,6554.567344,7548.022611,8.950630e+02,7057.720180,5071.373066,1081.903393,1521.703530,4101.358337,5988.463067
d_1919,53210.710811,16812.525729,14361.369668,22036.815415,3947.204166,8694.376904,6532.655433,2.862579e+03,4809.266181,7001.257417,2550.846070,4497.034664,6797.997994,5517.493071
d_1920,57251.137804,14469.669747,15525.059838,27256.408219,7037.208226,6545.999587,8125.382530,5.547818e+03,5690.945284,4788.618219,5045.496335,5693.077646,2400.692780,6375.899321
d_1921,45279.563834,17772.314206,9762.783331,17744.466297,6360.173697,5688.853247,4302.156844,1.393283e+03,3905.204201,1114.190154,4743.388976,4080.069967,6857.610844,6834.633395
d_1922,48099.315383,15553.070936,12864.786510,19681.457937,5581.061256,4280.113461,6425.602915,3.394680e+03,6834.809378,4137.740930,1892.236202,3109.928570,6335.587783,6107.554583
d_1923,41610.985301,11790.359851,11031.514786,18789.110664,5296.614595,3129.194670,7594.406636,2.768895e+03,5112.837359,2725.130539,3193.546888,2693.789767,4802.849178,4293.720906


### try hts with 28 observations with interwoven hierarchies

In [28]:
cat_sales = train.groupby('cat_id',as_index=False)[ts_cols].sum()
cat_sales['id_str'] = cat_sales['cat_id'] 
cat_sales = cat_sales[ ['id_str'] +  [c for c in cat_sales if c not in ['id_str']] ]
cat_sales = cat_sales.drop(['cat_id'],axis=1)

In [29]:
dept_sales = train.groupby('dept_id',as_index=False)[ts_cols].sum()
dept_sales['id_str'] = dept_sales['dept_id'] 
dept_sales = dept_sales[ ['id_str'] +  [c for c in dept_sales if c not in ['id_str']] ]
dept_sales = dept_sales.drop(['dept_id'],axis=1)

In [30]:
all_aggregates = pd.concat([all_sales,state_sales,store_sales,cat_sales,dept_sales],ignore_index=True)
agg_last_28d = all_aggregates.set_index('id_str')[all_aggregates.columns[-28:]].transpose()

In [31]:
df_last_28d = train[train.columns[-28:]].transpose()

In [32]:
df_last_28d_all_ts = pd.concat([agg_last_28d, df_last_28d], axis=1)

In [33]:
df_last_28d_all_ts.head()

,all,CA,TX,WI,CA_1,CA_2,CA_3,CA_4,TX_1,TX_2,...,FOODS_3_818_WI_3_evaluation,FOODS_3_819_WI_3_evaluation,FOODS_3_820_WI_3_evaluation,FOODS_3_821_WI_3_evaluation,FOODS_3_822_WI_3_evaluation,FOODS_3_823_WI_3_evaluation,FOODS_3_824_WI_3_evaluation,FOODS_3_825_WI_3_evaluation,FOODS_3_826_WI_3_evaluation,FOODS_3_827_WI_3_evaluation
d_1914,38793,17524,10662,10607,4472,3926,6359,2767,3076,3883,...,0,4,2,0,0,0,0,0,1,0
d_1915,35487,15012,9933,10542,3703,3525,5289,2495,2853,3502,...,4,2,2,0,2,0,1,0,3,0
d_1916,34445,14836,9575,10034,3715,3527,5065,2529,2984,3256,...,0,0,1,0,2,0,1,1,0,0
d_1917,34732,14664,9655,10413,3618,3754,5015,2277,2664,3441,...,1,0,1,3,2,2,1,1,1,0
d_1918,42896,17180,12162,13554,4573,4382,5705,2520,3687,4023,...,2,1,1,1,2,2,0,0,2,0


There are 30490 bottom level timeseries (store-product level); 7 department ts; 3 category ts, 
10 store ts, 3 states ts, 1 top ts (all) <br>
In total, 30514 ts

### manually create sum_mat

In [34]:
n_bottom_ts = 30490
row_all = [1]*n_bottom_ts

In [35]:
row_states = []
for state in ['CA', 'TX', 'WI']:
    row_state = [1 if state in i else 0 for i in list(df_last_28d.columns)]
    row_states.append(row_state)

In [36]:
sum([sum(i) for i in row_states])

30490

In [37]:
row_stores = []
for store in ['CA_1', 'CA_2', 'CA_3', 'CA_4', 'TX_1', 'TX_2',
       'TX_3', 'WI_1', 'WI_2', 'WI_3']:
    row_store = [1 if store in i else 0 for i in list(df_last_28d.columns)]
    row_stores.append(row_store)

In [38]:
sum([sum(i) for i in row_stores])

30490

In [39]:
row_cats = []
for cat in ['FOODS', 'HOBBIES', 'HOUSEHOLD']:
    row_cat = [1 if cat in i else 0 for i in list(df_last_28d.columns)]
    row_cats.append(row_cat)

In [40]:
sum([sum(i) for i in row_cats])

30490

In [41]:
row_depts = []
for dept in ['FOODS_1', 'FOODS_2', 'FOODS_3', 'HOBBIES_1', 'HOBBIES_2',
       'HOUSEHOLD_1', 'HOUSEHOLD_2']:
    row_dept = [1 if dept in i else 0 for i in list(df_last_28d.columns)]
    row_depts.append(row_dept)

In [42]:
sum([sum(i) for i in row_depts])

30490

In [43]:
row_agg = [row_all] + row_states + row_stores + row_cats + row_depts

In [44]:
len(row_agg)

24

In [45]:
sum_mat = np.append(row_agg, np.identity(n_bottom_ts), axis = 0)

In [46]:
sum_mat.shape

(30514, 30490)

In [47]:
sum_mat_labels = list(df_last_28d_all_ts.columns)

#### generate synthetic prediction data at all levels

In [48]:
std_all_ts = df_last_28d_all_ts.std()

In [49]:
# add noise to true sales numbers to get synthetic prediction data

noises = []
for i in range(df_last_28d_all_ts.shape[1]):
    mu, sigma = 0, std_all_ts[i]
    noise = np.random.normal(mu, sigma, [1, df_last_28d_all_ts.shape[0]])
    noises.append(noise)
noise_all = np.concatenate(noises, axis=0).transpose()

In [50]:
synthetic_pred_28_all_ts = df_last_28d_all_ts + noise_all

In [51]:
synthetic_pred_28_all_ts.head()

,all,CA,TX,WI,CA_1,CA_2,CA_3,CA_4,TX_1,TX_2,...,FOODS_3_818_WI_3_evaluation,FOODS_3_819_WI_3_evaluation,FOODS_3_820_WI_3_evaluation,FOODS_3_821_WI_3_evaluation,FOODS_3_822_WI_3_evaluation,FOODS_3_823_WI_3_evaluation,FOODS_3_824_WI_3_evaluation,FOODS_3_825_WI_3_evaluation,FOODS_3_826_WI_3_evaluation,FOODS_3_827_WI_3_evaluation
d_1914,39369.410737,18288.409562,9564.234382,12859.018097,3961.573589,4943.194490,7086.893034,3127.203138,3344.816799,3136.442232,...,2.574987,2.865972,0.506169,0.537456,-0.911758,0.731677,-0.477728,0.388479,2.061156,-1.651298
d_1915,36615.887770,16803.278941,8318.553835,7852.430312,2026.787293,2269.215679,5925.413212,2426.257102,2356.801232,2959.731594,...,5.833119,2.349885,1.130014,0.781888,2.753583,0.007082,1.085046,1.009048,6.123860,0.079292
d_1916,36802.287265,12358.795775,10719.870311,9571.602218,4509.489477,3897.272048,4920.354692,2407.997296,3047.460091,3247.772298,...,-0.491306,0.071433,3.020806,1.985542,1.667928,0.478749,1.298450,1.652849,0.612661,0.341853
d_1917,30815.984427,10639.334377,9583.971990,8779.649441,2908.144601,2969.272879,5968.553712,2207.121590,2778.790070,3844.950682,...,1.482467,-3.302730,2.491579,2.024981,4.672238,2.805284,0.807851,2.567162,-0.059655,1.564530
d_1918,47413.042905,20621.775113,11201.044614,14566.628688,4017.440205,4395.500480,4548.636356,2919.221274,3811.256930,3733.882422,...,4.733455,1.538893,3.771817,2.847684,2.043965,2.909198,0.182534,-0.181100,3.085641,0.026506


#### apply optimal reconciliation

In [52]:
pred_dict = collections.OrderedDict()
for label in sum_mat_labels:
    pred_dict[label] = pd.DataFrame(data=synthetic_pred_28_all_ts[label].values, columns=['yhat'])

In [53]:
%%time
# add non-negative constraint when reconciliate
hat_mat = hts.functions.y_hat_matrix(pred_dict)

CPU times: user 31.3 s, sys: 272 ms, total: 31.6 s
Wall time: 31.6 s


In [ ]:
%%time
b_mat = []
for i in range(28):
    b_mat.append(lsq_linear(sum_mat, hat_mat[i].flatten(), bounds=(0, np.inf))['x'])

In [ ]:
%%time
b_mat_final = np.stack(b_mat).transpose()
revised_nnls = np.dot(sum_mat, b_mat_final).transpose()
revised_forecasts_nnls = pd.DataFrame(data=revised_nnls,
                                 index=synthetic_pred_28_all_ts.index,
                                 columns=sum_mat_labels)